In [27]:
from pathlib import Path
patterns = Path('./patterns')
choices = list(patterns.glob("*"))
for i,pattern in enumerate(choices):
    print(i+1,pattern.name)

1 new_plugin_with_types
2 new_tool


In [28]:
choice = int(input(f"Choose a pattern ({1}-{len(choices)}): "))
assert 1 <= choice <= len(choices)
pattern = choices[choice-1]
print("You chose",pattern.name)

You chose new_plugin_with_types


In [30]:
pyfile = pattern / "__pattern__.py"
assert pyfile.exists(), f"{pyfile} does not exist"
import importlib
spec = importlib.util.spec_from_file_location("module.name", pyfile)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

# Check if the function exists, call it, and store the result
assert hasattr(module, 'gather_global_variables')
global_variables = module.gather_global_variables()
print(global_variables)

{'crate_name': 'benis', 'crate_name_pascal': 'Benis'}
